Run Iterations all at Once

In [18]:
import importlib

import metaspread
import metaspread.simrunner as simrunner

importlib.reload(metaspread)

import pandas as pd
import random
import time
import os
import ast
import json
import uuid

def run_n_times(number_of_simulations, maximum_steps):
    # Set random seed for reproducibility
    random.seed(int(time.time()))

    # Get the directory where metaspread is installed
    package_dir = os.path.dirname(metaspread.__file__)

    # Path to simulation_configs.csv inside metaspread
    config_path_default = os.path.join(package_dir, "default_configs.csv")

    # Load default parameters from the CSV file
    df_configs_default = pd.read_csv(config_path_default, header=0, converters={"Values": ast.literal_eval})
    default_params = dict(zip(df_configs_default["Names"], df_configs_default["Values"]))

    # List of parameters to vary
    user_selected_params = ["dM", "dE", "phiM","phiE","mesenchymal_proportion","epithelial_proportion"]  # Replace with actual parameter names
    valid_params = [param for param in user_selected_params if param in default_params]

    # Number of simulations
    num_simulations = number_of_simulations

    # Store all parameter variations
    simulation_logs = []

    # Loop over multiple iterations
    for i in range(num_simulations):
        print(f"Running simulation {i+1}/{num_simulations}...")

        simulation_id = str(uuid.uuid4())[:8]

        # Modify parameters randomly
        varied_params = default_params.copy()
        for param in valid_params:
            if param != "mesenchymal_proportion" and param != "epithelial_proportion":
                default_value = float(default_params[param])  # Ensure it's numeric
                new_value = default_value * random.uniform(1/3, 3)
                varied_params[param] = new_value
            else:
                varied_params[param] = random.uniform(0,1)

        # Ensure grid size is always set to 50
        varied_params["gridsize"] = 200

        # set mesenchymal_porpotion and ensure the proportions add up to 1
        mes_proportion = random.uniform(0,1)
        varied_params["mesenchymal_proportion"] = mes_proportion
        varied_params["epithelial_proportion"] = 1 - mes_proportion

        # Update the CSV file with modified parameters
        config_path_changed = os.path.join(package_dir, "simulations_configs.csv")
        df_configs_changed = pd.read_csv(config_path_changed, header=0, converters={"Values": ast.literal_eval})
        df_configs_changed.loc[df_configs_changed["Names"].isin(valid_params + ["gridsize"]), "Values"] = df_configs_changed["Names"].map(varied_params)
        df_configs_changed.to_csv(config_path_changed, index=False)

        # Run simulation
        simulation = simrunner.run_simulation(simulation_id, max_steps=maximum_steps, data_collection_period=10)

        # Store parameter variations
        simulation_logs.append({
            "simulation_id": simulation_id,
            "varied_parameters": {param: varied_params[param] for param in valid_params},
            "gridsize": 200
        })

        print(f"Completed simulation {i+1}/{num_simulations}.")

    # Save all parameters used in simulations
    with open("metaspread_simulation_logs.json", "w") as f:
        json.dump(simulation_logs, f, indent=4)

    print(f"All {num_simulations} simulations completed successfully.")


## Run
run_n_times({number of simulations}, {maximum steps})

In [19]:
run_n_times(1,1500)

Running simulation 1/1...

	 Folder for this simulation: Simulations/Sim-02f355e1-Date-2025-03-01_18-22-30
	 Saving agents data at: Simulations/Sim-02f355e1-Date-2025-03-01_18-22-30
	 Saving Mmp2 data at: Simulations/Sim-02f355e1-Date-2025-03-01_18-22-30/Mmp2
	 Saving Ecm data at: Simulations/Sim-02f355e1-Date-2025-03-01_18-22-30/Ecm
	 Saving Vasculature data at: Simulations/Sim-02f355e1-Date-2025-03-01_18-22-30/Vasculature
	 Saving all the simulations parameters at: Simulations/Sim-02f355e1-Date-2025-03-01_18-22-30/configs.csv
Starting simulation from zero!
Finished the simulation at time step 1500!
Completed simulation 1/1.
All 1 simulations completed successfully.


In [ ]:
import metaspread.datagenerator as datagenerator
import metaspread.graphgenerator as graphgenerator

In [26]:
simulation_folder = "Sim-6120b2be-Date-2025-03-01_16-54-53"
amount_of_pictures= 50

datagenerator.generate_data(simulation_folder)

	Analyzing data in the folder Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53

Using cells data at: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/CellsData.csv
Using Ecm data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Ecm
Using Mmp2 data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Mmp2
Using vasculature data at: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Vasculature
Max step reached is 500 while 500 was expected.

Saving tumor data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Data analysis/Tumor dynamics
Saving cells numbers data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Data analysis/Cells growth
Saving vasculature data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Data analysis/Vasculature dynamics

Grid: 1
	Saving tumor data...
	Saving cells numbers graph data...

Grid: 2
	Saving tumor data...
	Saving cells numbers graph data...

Grid: 3
	Saving tumor data...
	Saving

In [27]:
graphgenerator.generate_graphs(simulation_folder, amount_of_pictures)

Analyzing data in the folder Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53

Using cells data at: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/CellsData.csv
Using Ecm data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Ecm
Using Mmp2 data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Mmp2
Using vasculature data at: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Vasculature
Max step reached is 500 while 500 was expected.

Saving tumor images in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Graphical analysis/Tumor dynamics
Saving Mmp2 images in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Graphical analysis/Mmp2 dynamics
Saving Ecm images in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Graphical analysis/Ecm dynamics
Saving cells numbers images in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Graphical analysis/Cells growth
Saving vasculature images in the folder: Simulation